##Setting up the environment
###Importing libraries:

* os: This module provides a way to use operating system dependent functionality, like reading environment variables.
* OpenAI: This is the official OpenAI Python client, used to interact with OpenAI's API.
* requests: A popular library for making HTTP requests in Python.
* json: Used for parsing JSON data, which is common in API responses.
* logging: Provides a flexible framework for generating log messages in Python.
* getpass: Allows secure password prompts where the input is not displayed on the screen.


###Setting up logging:

* We use `logging.basicConfig()` to configure the logging system. The `level=logging.INFO` argument sets the threshold for logging messages to INFO level and above.
* We create a logger object named logger that we'll use throughout our script to log important information and errors.



This setup ensures we have all necessary tools to interact with APIs, handle data, and track any issues that might occur during execution.

In [1]:
!pip install openai
import os
from openai import OpenAI
import requests
import json
import logging
from getpass import getpass

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("Libraries imported and logging set up successfully.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
Libraries imported and logging set up successfully.


##Secure API key handling
###Secure API key input:

* We use `getpass()` to prompt the user for their API keys. This function hides the input, making it more secure than using regular input().
* This approach is safer than hardcoding API keys in your script, which could accidentally be shared or exposed.


###Setting environment variables:

* We use `os.environ` to set environment variables for both API keys.
* Environment variables are a secure way to store sensitive information, as they're not part of your code and are only accessible within the current process.


###Initializing the OpenAI client:

* We create an instance of the OpenAI client using the API key we just set.
* Using `os.getenv("OPENAI_API_KEY")` retrieves the API key from the environment variables.



This method ensures that your API keys are handled securely and are readily available for use in your script.

In [2]:
# Cell 2: Secure API key handling

# Securely input API keys
openai_api_key = getpass("Enter your OpenAI API key: ")
llumo_api_key = getpass("Enter your Llumo API key: ")

# Set environment variables
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['LLUMO_API_KEY'] = llumo_api_key

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("API keys securely set and OpenAI client initialized.")

Enter your OpenAI API key: ··········
Enter your Llumo API key: ··········
API keys securely set and OpenAI client initialized.


##Define Llumo compression function
###Function definition:

* We define a function `compress_with_llumo` that takes a text input and an optional topic.


###API setup:

* We retrieve the Llumo API key from environment variables.
* We set the API endpoint and prepare headers for the HTTP request.


###Payload preparation:

* We create a payload dictionary with the input text.
* If a topic is provided, we add it to the payload.


###API request:

* We use `requests.post()` to send a POST request to the Llumo API.
`response.raise_for_status()` will raise an exception for HTTP errors.


###Response parsing:

* We parse the JSON response and extract the compressed text and token counts.
* We calculate the compression percentage.


###Error handling:

* We use a try-except block to catch potential errors:

 * JSON decoding errors
 * Request exceptions
 * Unexpected response structure


* If an error occurs, we log it and return the original text with failure indicators.


### Return values:

* The function returns a tuple containing:

 * Compressed text (or original if compression failed)
 * Success boolean
 * Compression percentage
 * Initial token count
 * Final token count





This function encapsulates the entire process of interacting with the Llumo API for text compression, including error handling and result processing.

In [3]:
def compress_with_llumo(text):
    # Retrieve the Llumo API key from environment variables
    LLUMO_API_KEY = os.getenv('LLUMO_API_KEY')

    # Define the Llumo API endpoint for text compression
    LLUMO_ENDPOINT = "https://app.llumo.ai/api/compress"

    # Set the headers for the API request
    headers = {
        "Content-Type": "application/json",  # Specify the content type as JSON
        "Authorization": f"Bearer {LLUMO_API_KEY}"  # Authorization header with the API key
    }

    # Create the payload for the API request with the provided text
    payload = {"prompt": text}

    try:
        # Make a POST request to the Llumo API with the payload and headers
        response = requests.post(LLUMO_ENDPOINT, json=payload, headers=headers)
        response.raise_for_status()

        # Parse the JSON response from the API
        result = response.json()
        data = json.loads(result['data']['data'])

        # Extract compressed text and token information
        compressed_text = data.get('compressedPrompt', text)
        initial_tokens = data.get('initialTokens', 0)
        final_tokens = data.get('finalTokens', 0)

        # Calculate the compression percentage
        compression_percentage = ((initial_tokens - final_tokens) / initial_tokens) * 100 if initial_tokens else 0

        # Return the compressed text, success flag, compression percentage, and token counts
        return compressed_text, True, compression_percentage, initial_tokens, final_tokens
    except Exception as e:
        print(f"Error compressing text: {str(e)}")
        return text, False, 0, 0, 0


##Define example prompt and test without compression
###Defining the prompt:

* We create a detailed prompt about photosynthesis. This serves as our example text for compression.


###Testing without compression:

* We use the OpenAI client to send a request to the GPT-3.5-turbo model.
* The messages parameter follows the chat format:

 * A system message sets the AI's role.
 * A user message contains our prompt.




###Displaying results:

* We print the AI's response to the prompt.
* We also print the total number of tokens used, which is important for understanding API usage and costs.

This cell demonstrates how the API would typically be used without any compression, providing a baseline for comparison.



In [6]:
# Cell 4: Define example prompt and test without compression

# Example prompt
topic = "Binary Search Trees (BST)"
field = "Data Structures and Algorithms (DSA)"
difficulty_level = "mid-level"
experience_range = "2-4 years"
time_limit = "45-60 minute"
key_operations = "insertion, deletion, search, and traversal"
algorithm_focus = "search and traversal algorithms"
advanced_concept = "self-balancing trees"
core_operations = "insertion, deletion, and search"
complexity_analysis = "time and space complexity"
notation = "Big-O notation"
scenario_types = "best-case, average-case, and worst-case"
edge_cases = "empty trees, duplicate values, and highly unbalanced trees"
optimization_technique = "implementing a self-balancing tree"
specific_edge_cases = "empty trees or duplicate values"
programming_language = "Python"
advanced_operation = "maintaining subtree sizes for order statistics"
comparison_structure_1 = "heaps"
comparison_structure_2 = "hash tables"

prompt = f"""
Create an advanced and highly detailed interview question centered on {topic}, a fundamental concept in {field}. The objective of this question is to thoroughly evaluate the candidate's skills in implementing, optimizing, and analyzing algorithms related to {topic}. This question should be suitable for a {difficulty_level} software engineer with {experience_range} of industry experience. The question should be challenging yet solvable within a {time_limit} interview. Below are the specific requirements and guidelines that should be followed:

Core Focus Areas:
Data Structure: The question must revolve around {topic}. It should test the candidate's understanding of how {topic} operate, including their properties and the key operations performed on them, such as {key_operations}.
Algorithm Type: Emphasize {algorithm_focus} within the context of {topic}. The candidate should demonstrate proficiency in implementing these algorithms and understanding their implications on the overall structure and efficiency of the {topic}.
Advanced Concepts: The question should also touch upon {advanced_concept} as a potential extension, assessing the candidate's knowledge of how these advanced structures maintain the efficiency of operations in the presence of skewed data.
Difficulty Level and Target Audience:
The question should be of {difficulty_level} difficulty, appropriate for candidates with {experience_range} of professional experience in software development, particularly those who have a solid understanding of data structures and algorithms.
Ensure that the problem is challenging enough to differentiate between candidates who have merely theoretical knowledge and those who can apply this knowledge in a practical, optimized manner.
The problem should be solvable within a {time_limit} interview, but it should also allow for deeper discussion on optimizations, edge cases, and alternative approaches if time permits.
Assessment Criteria:
Implementation: The candidate should be required to implement core operations of a {topic} such as {core_operations}. This implementation should be robust, efficient, and correct.
Algorithm Optimization: Evaluate how well the candidate can optimize these operations, particularly in scenarios where the {topic} may become unbalanced. Encourage discussion around the trade-offs involved in implementing different balancing techniques or using a self-balancing tree.
Complexity Analysis: Require the candidate to analyze the {complexity_analysis} of their implementation. They should be able to articulate the complexity in terms of {notation}, explaining how their implementation performs in {scenario_types} scenarios.
Edge Cases and Robustness: The question should explicitly require handling of edge cases such as {edge_cases}. The candidate should demonstrate an understanding of how these cases can affect the behavior and efficiency of {topic} operations.
Problem-Solving Approach: The candidate's approach to solving the problem should be methodical, logical, and well-structured. They should clearly explain their thought process, justify their choices, and consider alternative solutions.
Question Structure:
Part 1: Basic Implementation: Start with a task requiring the candidate to implement basic operations of a {topic}. This part should test their fundamental understanding of how {topic} work and ensure they can write correct and efficient code.
Part 2: Optimization Challenge: Once the basic implementation is complete, present the candidate with a challenge to optimize their {topic}. For instance, they could be asked to handle scenarios where the {topic} becomes unbalanced, discussing potential solutions like {optimization_technique}.
Part 3: Edge Cases and Analysis: Ask the candidate to identify and address specific edge cases. They should explain how their implementation handles cases like {specific_edge_cases} and how it affects the overall performance.
Part 4: Complexity Discussion: End with a discussion on complexity. The candidate should analyze the time and space complexity of their implementation, considering different scenarios and providing a clear explanation for their conclusions.
Detailed Solution Expectations:
Step-by-Step Implementation: The candidate should write code in {programming_language}, with each step clearly documented and explained. The code should be clean, efficient, and follow best practices.
Explanatory Commentary: Each part of the code should be accompanied by detailed comments explaining what the code is doing and why certain choices were made. This should include the rationale behind the data structures used, the logic of the algorithms, and any optimizations implemented.
Complexity Justification: For each operation implemented, the candidate should provide a thorough analysis of the time and space complexity. They should explain how the complexity is derived, using {notation}, and discuss how it varies under different conditions (e.g., balanced vs. unbalanced trees).
Edge Case Handling: The solution should be robust, with explicit handling of edge cases. The candidate should describe how they tested for these cases and the adjustments they made to ensure their code handles them correctly.
Optimizations and Alternatives: If applicable, the candidate should be encouraged to suggest alternative approaches or optimizations. For example, they could discuss the pros and cons of using a {advanced_concept} versus a standard {topic}, and under what circumstances one might be preferred over the other.
Bonus Components:
Advanced Tree Operations: If time allows, include a bonus question that challenges the candidate to implement more advanced operations on the {topic}, such as {advanced_operation}. This would test their ability to extend their implementation and handle more complex requirements.
Theoretical Discussion: Encourage a discussion on the theory behind {topic}, including their place in the broader context of data structures and algorithms. Ask the candidate to compare {topic} with other data structures like {comparison_structure_1} or {comparison_structure_2}, explaining when and why a {topic} might be preferable.
Real-World Application: Finally, ask the candidate to think about real-world applications of {topic}. How might a {topic} be used in a production environment? What are some potential pitfalls, and how could they be mitigated?
Evaluation and Scoring:
Correctness: Assess the correctness of the candidate's implementation. Did they correctly implement the required operations? Does their code handle all specified cases, including edge cases?
Efficiency: Evaluate the efficiency of the candidate's solution. Did they choose the most appropriate algorithms and data structures for the problem? How does their implementation perform in terms of time and space complexity?
Clarity and Communication: Consider how well the candidate communicated their thought process. Were they able to clearly explain their approach, justify their choices, and articulate the complexity of their solution?
Problem-Solving Skills: Finally, assess the candidate's overall problem-solving skills. Did they approach the problem in a logical and structured way? Were they able to adapt their solution to handle more complex requirements or edge cases? How well did they perform under time constraints?
"""



print("Without Llumo Compression:")
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)
print(response.choices[0].message.content)
print(f"Tokens used: {response.usage.total_tokens}\n")

Without Llumo Compression:
**Interview Question:**
---
**Part 1: Basic Implementation**

You are given the task of implementing basic operations for a Binary Search Tree (BST) in Python. Your implementation should include methods for insertion, deletion, and search on the BST. 

Please write Python code for the following:
1. Implement a BST class with the necessary methods for insertion, deletion, and search.
2. Ensure that your implementation handles cases of duplicate values appropriately.
3. Write a simple test script to demonstrate the functionality of your BST class with a sample insertion, deletion, and search operations.

Please provide detailed comments in your code to explain your thought process and choices made during implementation.

---

**Part 2: Optimization Challenge**

Once you have completed the basic implementation, let's focus on optimization. Discuss a scenario where the BST may become unbalanced, significantly impacting the efficiency of search operations. Please 

##Test with Llumo compression
###Applying Llumo compression:

* We call compress_with_llumo() with our example prompt.
* The function returns multiple values, which we unpack into separate variables.


###Checking compression success:

* We use an if statement to check if compression was successful.
* If successful, we print compression statistics: percentage, initial and final token counts.


###Using the compressed prompt:

* If compression succeeded, we use the compressed prompt in our API call to GPT-3.5-turbo.
* We use the same message structure as before, but with the compressed prompt.


###Displaying results:

* We print the AI's response to the compressed prompt.
* We print the number of tokens used with the compressed prompt.


###Handling compression failure:

* If compression fails, we print a message indicating this.
* In a real application, you might want to fall back to using the original prompt in this case.



This cell demonstrates the full process of compressing a prompt with Llumo and using it with the OpenAI API. By comparing the results and token usage with the previous cell, you can see the potential benefits of using Llumo compression in terms of token efficiency and cost savings.

In [7]:
# Cell 5: Test with Llumo compression

print("With Llumo Compression:")
compressed_prompt, success, compression_percentage, initial_tokens, final_tokens = compress_with_llumo(prompt)

if success:
    print(f"Compression achieved: {compression_percentage:.2f}%")
    print(f"Initial tokens: {initial_tokens}, Final tokens: {final_tokens}")

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": compressed_prompt}
        ]
    )
    print(response.choices[0].message.content)
    print(f"Tokens used: {response.usage.total_tokens}")
else:
    print("Compression failed. Using original prompt.")
    # Use the original prompt if compression fails

With Llumo Compression:
Compression achieved: 50.25%
Initial tokens: 1389, Final tokens: 691
Interview Question: Implementing and Optimizing Binary Search Trees (BSTs) - A Deep Dive

Part 1: Basic Task Implementation (20 minutes)
You are tasked with implementing a Binary Search Tree (BST) in Python. Your implementation should include the core operations of insertion, deletion, and search. You should strive for efficiency and correctness in your implementation. Handle scenarios where the BST may become unbalanced and discuss potential solutions for implementing a self-balancing tree. Additionally, consider cases such as handling duplicate keys and empty trees in your implementation for robustness and performance.

Part 2: Complex Discussion (25 minutes)
After completing the basic implementation, let's move on to a deeper discussion. Explain the time and space complexity of your implementation. Consider scenarios where the BST becomes unbalanced and discuss methods to optimize the tree t